In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import glob                # 파일을 읽어오는 작업에 용이
import os
import re                  # 정규표현식을 위한 Regex 지원 모듈 (문장 데이터를 정돈) 
import numpy as np         # 변환된 문장 데이터(행렬) 처리
import tensorflow as tf    # 텐서플로우

In [ ]:
txt_file_path = '/content/drive/MyDrive/Colab Notebooks/lyrics.txt'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187052
Examples:
 ["Busted flat in Baton Rouge, waitin' for a train", "And I's feelin' near as faded as my jeans", 'Bobby thumbed a diesel down, just before it rained']


In [ ]:
# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
# 이 순서로 처리해주면 문제가 되는 상황을 방지할 수 있겠네요!
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

# 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


In [ ]:
# 여기에 정제된 문장을 모을겁니다
corpus = []
for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    if sentence[-1] == "]": continue # ']'로 끝나는 파트를 나눈 문장은 건너뜁니다
    if sentence[-1] == ")": continue # ')'로 끝나는 코러스 문장은 건너뜁니다    
    
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
corpus[:5]

['<start> busted flat in baton rouge , waitin for a train <end>',
 '<start> and i s feelin near as faded as my jeans <end>',
 '<start> bobby thumbed a diesel down , just before it rained <end>',
 '<start> it rode us all the way to new orleans i pulled my harpoon out of my dirty red bandanna <end>',
 '<start> i was playin soft while bobby sang the blues , yeah <end>']

In [ ]:
# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용합니다
def tokenize(corpus):
    # 7000단어를 기억할 수 있는 tokenizer를 만들겁니다
    # 우리는 이미 문장을 정제했으니 filters가 필요없어요
    # 7000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=7000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen = 15)  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2 3730 1621 ...    0    0    0]
 [   2    8    5 ...    0    0    0]
 [   2  794    1 ...    0    0    0]
 ...
 [   5   22  691 ... 4424    4    3]
 [   5   22  691 ... 4424    4    3]
 [   5   22  691 ... 4424   21    3]] <keras_preprocessing.text.Tokenizer object at 0x7fe01e38b190>


In [ ]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[   2 3730 1621   14 6883 4045    4 1110   28    9  712    3    0    0]
[3730 1621   14 6883 4045    4 1110   28    9  712    3    0    0    0]


In [ ]:
from sklearn.model_selection import train_test_split
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.20, random_state=40)

In [ ]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset element_spec=(TensorSpec(shape=(256, 14), dtype=tf.int32, name=None), TensorSpec(shape=(256, 14), dtype=tf.int32, name=None))>

In [ ]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 1024
hidden_size = 4096
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [ ]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법입니다.
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

<tf.Tensor: shape=(256, 14, 7001), dtype=float32, numpy=
array([[[ 1.3457803e-04,  2.3079510e-05,  2.8311345e-04, ...,
         -2.9986564e-04, -1.0709953e-04,  4.4410172e-04],
        [-3.7106659e-04, -3.3608772e-04,  9.2680915e-04, ...,
         -8.8521738e-05,  1.7184802e-04,  1.0215031e-03],
        [-1.1021886e-03, -5.9681042e-04,  8.5007842e-04, ...,
         -7.8712306e-05, -1.6643060e-04,  1.4850395e-03],
        ...,
        [-2.6316252e-03,  5.6448444e-05, -9.8276313e-04, ...,
         -1.2014246e-03, -4.1169897e-04,  7.0818816e-04],
        [-2.0441762e-03,  3.6292864e-04, -1.1253910e-03, ...,
         -6.2584644e-04, -2.6437751e-04,  1.3278209e-04],
        [-1.4444221e-03,  6.1357615e-04, -1.2892330e-03, ...,
         -4.8873328e-05, -1.1744445e-04, -4.6641342e-04]],

       [[ 1.3457803e-04,  2.3079510e-05,  2.8311345e-04, ...,
         -2.9986564e-04, -1.0709953e-04,  4.4410172e-04],
        [ 2.9666157e-04,  4.6182511e-05,  3.3561213e-04, ...,
         -3.6709076e-05, -

In [ ]:
model.summary()

Model: "text_generator_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_9 (Embedding)     multiple                  7169024   
                                                                 
 lstm_18 (LSTM)              multiple                  83902464  
                                                                 
 lstm_19 (LSTM)              multiple                  134234112 
                                                                 
 dense_9 (Dense)             multiple                  28683097  
                                                                 
Total params: 253,988,697
Trainable params: 253,988,697
Non-trainable params: 0
_________________________________________________________________


In [ ]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=7)

lyricist = TextGenerator(tokenizer.num_words + 1, embedding_size, hidden_size)


Epoch 1/7
650/650 [==============================] - 455s 696ms/step - loss: 3.3623
Epoch 2/7
650/650 [==============================] - 453s 696ms/step - loss: 2.7685
Epoch 3/7
650/650 [==============================] - 452s 696ms/step - loss: 2.4580
Epoch 4/7
650/650 [==============================] - 453s 696ms/step - loss: 2.1454
Epoch 5/7
650/650 [==============================] - 452s 696ms/step - loss: 1.8266
Epoch 6/7
650/650 [==============================] - 453s 696ms/step - loss: 1.5250
Epoch 7/7
650/650 [==============================] - 453s 696ms/step - loss: 1.2743


In [ ]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=15):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   
        # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

In [ ]:
generate_text(lyricist, tokenizer, init_sentence="<start> i love", max_len=15)

'<start> i love frauds peeps peeps thirteen dimension dimension dimension such feast feast hid hid '